# 运行

几种输出方式：流式输出、同时回答多个 instruction、考虑上一轮对话、交互式输出

## 流式输出

不同于将一整段回答直接输出，流式输出是将模型输出过程中的每一小步/部分内容作为一个个事件，每个事件可能表示几个字或图片或其他，通过控制事件达到流式输出

run_streamed() 不能使用 await

In [ ]:
from agents import Agent, Runner, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
from openai.types.responses import ResponseTextDeltaEvent
import os
import asyncio
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"
set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

agent = Agent(
    name="天气助手",
    instructions="始终用汉赋的形式回答用户",
    model=llm,
)

async def main():
    result = Runner.run_streamed(agent, "给我讲个程序员相亲的笑话")
    async for event in result.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            # 只关心指定类型的事件
            print(event.data.delta, end="", flush=True)

await main()

## 列表输入

messages 可以理解为将多个 instructions 拼接到一个列表中

In [ ]:
import asyncio
from agents.extensions.models.litellm_model import LitellmModel
from agents import Agent, Runner, set_tracing_disabled
import os
import asyncio
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"
set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

agent = Agent(
    name="小助手",
    instructions="回答用户的问题。",
    model=llm,
)

messages = [
    {"role": "user", "content": "孔子的全名叫什么?"},
    {"role": "user", "content": "孟子的全名叫什么?"},
    ]

async def main():
    result = await Runner.run(
        agent,
        input=messages,
    )
    print(result.final_output)

await main()

## 连续多轮对话

将**之前的** instruction 及回答合并到下一个 instruction 中，实现考虑上下文的作用

In [ ]:
import asyncio
from agents.extensions.models.litellm_model import LitellmModel
from agents import Agent, Runner, set_tracing_disabled
import os
import asyncio
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"
set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

agent = Agent(name="Assistant", model=llm, instructions="Reply very concisely.")

async def main():
    # First turn
    result = await Runner.run(agent, "What city is the Golden Gate Bridge in?")
    print(result.final_output)
    # San Francisco
    print("firt result.to_input_list()", result.to_input_list())

    # Second turn
    new_input = result.to_input_list() + [{"role": "user", "content": "What state is it in?"}]
    result = await Runner.run(agent, new_input)
    print(result.final_output)

    print("second result.to_input_list()", result.to_input_list())

    # Third
    new_input_ = result.to_input_list() + [{"role": "user", "content": "which bridge have we mentioned?"}]
    result = await Runner.run(agent, new_input_)
    print(result.final_output)

    print("third result.to_input_list()", result.to_input_list())


await main()

## 终端的人类交互

run_demo_loop 在循环中提示用户输入，保留回合之间的对话历史，可以实现连续多轮对话同样的效果

默认情况下，它在生成模型输出时进行流式传输

In [ ]:
import asyncio
from agents.extensions.models.litellm_model import LitellmModel
from agents import Agent, Runner, set_tracing_disabled, run_demo_loop
import os
import asyncio
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"
set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)
agent = Agent(name="Assistant", model=llm, instructions="Reply very concisely.")

async def main():
    await run_demo_loop(agent)

await main()